### This part is hovernet preprocess: After MATLAB part of analysis finished on the MATLAB code, this is for modifying some of the wsi(ndpi)src, roisrc, and ndpisrc so that we're only using the 214 image files that we want to use. The MATLAB all_output_concat was manually made by just copy and pasting contents of the each of the excel output files from the MATLAB workflow. After running hovernet workflow, manually add hovernet result to all_output_concat to match 1090 features.
### These src's are used in the hovernet_json2df.py, just making sure all the src's are correct so that hovernet_json2df.py runs properly!

In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import shutil
import pickle
from tqdm import tqdm

In [ ]:
xlsrc = r'\\shelter\Kyu\skin_aging\clue_cohort\CLUE_image_list_230207_v2.xlsx'
ndpisrc = r'\\shelter\Kyu\skin_aging\clue_cohort\wsi'
xl = pd.read_excel(xlsrc)
xl_tmp = xl[xl["student score"] > 1]
wsi_list = xl_tmp.filename
# create ndpi path:
wsi_ndpi_list = [os.path.join(ndpisrc, x) for x in wsi_list]
dst_src = r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\desired_wsi'
for filename in wsi_ndpi_list:
    shutil.copy(filename, dst_src)
# create dl path:
dlsrc = r'\\shelter\Kyu\skin_aging\clue_cohort\DLmask1um'
wsi_list = [x.replace(".ndpi", ".tif") for x in wsi_list]
wsi_dl_list = [os.path.join(dlsrc, x) for x in wsi_list]
dst_src1 = r'\\shelter\Kyu\skin_aging\clue_cohort\DLmask1um\desired_DLmask'
for filename in wsi_dl_list:
    shutil.copy(filename, dst_src1)
# create roi path:
roisrc = r'\\shelter\Kyu\skin_aging\clue_cohort\annotations\roi\labeledmask_v2_021723'
wsi_list = [x.replace(".tif", ".png") for x in wsi_list]
wsi_roi_list = [os.path.join(roisrc, x) for x in wsi_list]
dst_src2 = r'\\shelter\Kyu\skin_aging\clue_cohort\annotations\roi\labeledmask_v2_021723\desired_roi'
for filename in wsi_roi_list:
    shutil.copy(filename, dst_src2)

### This part is hovernet postprocess: This code belows processes the generated .pkl file from hovernet to the .xlsx file in a form that we desire:

In [ ]:
json = pd.read_json(r"\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\json\2022-06-07 13.18.40.json", orient='index')
json = pd.DataFrame(json[0].loc['nuc']).T.drop(columns=['type_prob'])
json = json[json['contour'].map(len) > 5].reset_index(drop=True)
json

In [ ]:
bbox_tmp = json["bbox"].tolist()
for idx in range(len(bbox_tmp)):
    bbox_tmp[idx] = [[int(x/2),int(y/2)] for (x,y) in bbox_tmp[idx]]
json["bbox"] = bbox_tmp

In [ ]:
contour_tmp = json["centroid"].tolist()
contour_x1 = [contour_tmp[i][0]/2 for i in range(len(contour_tmp))]
contour_y1 = [contour_tmp[i][1]/2 for i in range(len(contour_tmp))]
new_contour_xy = list(zip(contour_x1,contour_y1))
new_contour_xy = [[new_contour_xy[i][0],new_contour_xy[i][1]] for i in range(len(new_contour_xy))] #,int(new_contour_xy[i][1])
json["centroid"] = new_contour_xy

In [ ]:
contour_tmp = json["contour"].tolist()
for idx in range(len(contour_tmp)):
    contour_tmp[idx] = [[int(x/2),int(y/2)] for (x,y) in contour_tmp[idx]]
json["contour"] = contour_tmp

In [ ]:
json

In [ ]:
import pandas as pd
pkl = pd.read_pickle(r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\df\2022-06-07 13.18.40.pkl')

In [ ]:
pkl

In [ ]:
bbox_tmp = pkl["bbox"].tolist()
for idx in range(len(bbox_tmp)):
    bbox_tmp[idx] = [[int(x/2),int(y/2)] for (x,y) in bbox_tmp[idx]]
pkl["bbox_20x"] = bbox_tmp
pkl['centroid_20x'] = pkl['centroid'].apply(lambda row: [_/2 for _ in row])
contour_tmp = pkl["contour"].tolist()
for idx in range(len(contour_tmp)):
    contour_tmp[idx] = [[int(x/2),int(y/2)] for (x,y) in contour_tmp[idx]]
pkl["contour_20x"] = contour_tmp

In [2]:
import os
import pickle
from tqdm import tqdm
import pandas as pd
import numpy as np

In [3]:
pkl_src=r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\df'
output_src=r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\df\output_excel'
pkl_list = [x for x in os.listdir(pkl_src) if x.endswith(".pkl")]
pkl_full_list = [os.path.join(pkl_src,x) for x in pkl_list]

In [ ]:
for idx, pkl in tqdm(enumerate(range(len(pkl_list))),desc="Pickle File Processed",colour='red',total = len(pkl_list)):
    with open(pkl_full_list[idx], 'rb') as f:
        print("converting pkl to df:")
        df = pickle.load(f)
        bbox_tmp = df["bbox"].tolist()
        for idx1 in range(len(bbox_tmp)):
            bbox_tmp[idx1] = [[int(x/2),int(y/2)] for (x,y) in bbox_tmp[idx1]]
        df["bbox_20x"] = bbox_tmp
        df['centroid_20x'] = df['centroid'].apply(lambda row: [_/2 for _ in row])

        contour_tmp = df["contour"].tolist()
        for idx2 in range(len(contour_tmp)):
            contour_tmp[idx2] = [[int(x/2),int(y/2)] for (x,y) in contour_tmp[idx2]]
        df["contour_20x"] = contour_tmp
    # df dict, key is the type and the value is a DataFrame with the relevant rows
    dfs_by_type = {}
    for t in range(2, 12):
        dfs_by_type[t] = df[df['type'] == t].reset_index(drop=True)

    expanded_dfs = []
    for t, df_t in dfs_by_type.items():
        new_cols = {f"{col}_type{t}": df_t[col] for col in df_t.columns if col != 'type'}
        expanded_dfs.append(pd.DataFrame(new_cols))

    expanded_df = pd.concat(expanded_dfs, axis=1)
    imID,_ = os.path.splitext(pkl_list[idx])
    output_pth = os.path.join(output_src,imID + ".xlsx")
    # save original df
    expanded_df.to_excel(output_pth, index=False)

    print("calculating average")
    # todo: get the distances!

    for col in expanded_df.columns:
        if expanded_df[col].dtype not in [int, float]:
            if col == "inroi_type2":
                continue
            expanded_df.drop(columns=col,inplace=True)
    expanded_df = expanded_df.replace([np.inf, -np.inf], np.nan).dropna()

    # drop cols with inroi in them
    inroi_cols = [col for col in expanded_df.columns if 'inroi_type' in col]
    expanded_df = expanded_df.drop(columns=inroi_cols)

    # groupby and average now:
    expanded_df = expanded_df.replace([np.inf, -np.inf, np.nan], np.nan)
    grouped = expanded_df.groupby('inroi_type2')
    grouped_list = [grouped.get_group(x) for x in grouped.groups]

    finaldf = pd.DataFrame()
    for idx in range(len(grouped_list)):
        groupdf = pd.DataFrame(grouped_list[idx].mean(skipna=True)).T
        finaldf = pd.concat([finaldf,groupdf])

    # save average df
    with pd.ExcelWriter(output_pth, engine='openpyxl', mode='a') as writer:
        finaldf.to_excel(writer, sheet_name='averages', index=False)

In [4]:
with open(pkl_full_list[1], 'rb') as f:
    print("converting pkl to df:")
    df = pickle.load(f)
    bbox_tmp = df["bbox"].tolist()
    for idx1 in range(len(bbox_tmp)):
        bbox_tmp[idx1] = [[int(x/2),int(y/2)] for (x,y) in bbox_tmp[idx1]]
    df["bbox_20x"] = bbox_tmp
    df['centroid_20x'] = df['centroid'].apply(lambda row: [_/2 for _ in row])

    contour_tmp = df["contour"].tolist()
    for idx2 in range(len(contour_tmp)):
        contour_tmp[idx2] = [[int(x/2),int(y/2)] for (x,y) in contour_tmp[idx2]]
    df["contour_20x"] = contour_tmp
# df dict, key is the type and the value is a DataFrame with the relevant rows
dfs_by_type = {}
for t in range(2, 12):
    dfs_by_type[t] = df[df['type'] == t].reset_index(drop=True)

expanded_dfs = []
for t, df_t in dfs_by_type.items():
    new_cols = {f"{col}_type{t}": df_t[col] for col in df_t.columns if col != 'type'}
    expanded_dfs.append(pd.DataFrame(new_cols))

expanded_df = pd.concat(expanded_dfs, axis=1)

converting pkl to df:


In [4]:
expanded_df

,bbox_type2,centroid_type2,contour_type2,inroi_type2,resident_area_type2,c2t_distance_type2,Area_type2,Perimeter_type2,Circularity_type2,MA_type2,...,Extent_type11,EquiDia_type11,imID_type11,dist2nearest_type11,oriA_type11,oriB_type11,local_align_type11,bbox_20x_type11,centroid_20x_type11,contour_20x_type11
0,"[[47823, 34526], [47834, 34538]]","[47843.716894977166, 34516.93378995434]","[[47839, 34511], [47839, 34511], [47838, 34511...",4,57357,"[71104.0, 70080.0, 67968.0, 67648.0, 65440.0, ...",51.5,36.569541,0.483925,14.607458,...,0.418182,7.653040,2022-06-07 13.29.45162897,8.062258,100.230293,121.238243,0.466269,"[[23124, 19143], [23129, 19148]]","[23241.87684729064, 19030.774630541873]","[[23243, 19028], [23242, 19028], [23242, 19028..."
1,"[[48206, 31595], [48226, 31615]]","[49013.318181818184, 30808.53229665072]","[[49013, 30798], [49013, 30799], [49011, 30799...",4,57357,"[68800.0, 68352.0, 65856.0, 65600.0, 62720.0, ...",303.0,68.404541,0.813734,20.563667,...,0.527778,4.918491,2022-06-07 13.29.45162897,70.066040,147.179016,115.431717,0.132018,"[[32004, 13341], [32007, 13344]]","[31774.84677419355, 13573.771505376344]","[[31774, 13572], [31774, 13572], [31773, 13572..."
2,"[[48234, 31555], [48253, 31572]]","[48971.41365046536, 30835.81385729059]","[[48971, 30826], [48970, 30827], [48969, 30827...",4,57357,"[68864.0, 68416.0, 65920.0, 65728.0, 62816.0, ...",234.0,63.378203,0.732059,19.714821,...,0.367521,7.399277,2022-06-07 13.29.45162897,18.973666,47.551579,48.870728,0.177923,"[[36719, 12238], [36723, 12245]]","[36817.528894472365, 12145.604271356784]","[[36820, 12143], [36820, 12144], [36819, 12144..."
3,"[[48239, 31602], [48251, 31612]]","[49015.17877906977, 30836.77034883721]","[[49015, 30831], [49015, 30831], [49014, 30831...",4,57357,"[68800.0, 68352.0, 65856.0, 65600.0, 62720.0, ...",79.0,40.064495,0.618469,11.910135,...,0.514620,10.585135,2022-06-07 13.29.45162897,26.655206,43.699333,48.536812,0.473651,"[[38639, 13372], [38648, 13376]]","[38461.992875647666, 13555.64378238342]","[[38463, 13551], [38463, 13551], [38462, 13551..."
4,"[[48253, 31598], [48268, 31611]]","[49012.431677018634, 30852.905279503106]","[[49011, 30845], [49010, 30846], [49009, 30846...",4,57357,"[68800.0, 68352.0, 65856.0, 65600.0, 62752.0, ...",149.5,47.963990,0.816621,15.649416,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57351,"[[93268, 27769], [93287, 27775]]","[93307.88819875776, 27742.231366459626]","[[93307, 27732], [93307, 27733], [93306, 27734...",3,141027,"[23440.0, 23024.0, 22720.0, 22784.0, 23728.0, ...",71.5,44.242641,0.459022,19.422844,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57352,"[[93282, 27542], [93289, 27558]]","[93086.25192307692, 27749.084615384614]","[[93080, 27746], [93080, 27746], [93079, 27746...",3,141027,"[23056.0, 22640.0, 22336.0, 22400.0, 23344.0, ...",54.5,43.899495,0.355375,16.125137,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57353,"[[93288, 27569], [93295, 27580]]","[93111.32180851063, 27755.172872340427]","[[93111, 27752], [93109, 27754], [93107, 27754...",3,141027,"[23088.0, 22672.0, 22384.0, 22432.0, 23392.0, ...",39.0,27.721349,0.637743,10.299442,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57354,"[[93293, 27556], [93302, 27563]]","[93095.54838709677, 27761.65860215054]","[[93095, 27757], [93092, 27760], [93092, 27763...",3,141027,"[23056.0, 22656.0, 22352.0, 22416.0, 23360.0, ...",42.5,27.135563,0.725306,9.056720,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
c2t_distance_cols = expanded_df.filter(regex='c2t_distance').columns
col_list = list(c2t_distance_cols)
all_df = pd.DataFrame()
for col in c2t_distance_cols:
    print("processing column: {}".format(col))
    counter = col_list.index(str(col)) + 1 # 11 columns
    tmp_df = expanded_df[[col]]
    tmp_df = tmp_df[col]
    tmp_df = tmp_df.to_frame() # series to df
    for i in range(2, 13):
        col_name = str(col) + "_" + str(i)
        tmp_df[col_name] = '' # or premake all the columns
    for id, row in tmp_df.iterrows(): # id: 0~57356
        for idx, value in enumerate(row): # idx: 0~10
            print(value[0])
            if isinstance(value,str):
                continue
            if np.any(pd.isnull(value)):
                tmp_df.iloc[id,(counter*(idx+1))] = np.nan
            else:
                tmp_df.iloc[id,(counter*(idx+1))] = value[idx]
    all_df = pd.concat([all_df,tmp_df])

processing column: c2t_distance_type2
71104.0


IndexError: invalid index to scalar variable.

In [10]:
tmp_df

,c2t_distance_type2,c2t_distance_type2_2,c2t_distance_type2_3,c2t_distance_type2_4,c2t_distance_type2_5,c2t_distance_type2_6,c2t_distance_type2_7,c2t_distance_type2_8,c2t_distance_type2_9,c2t_distance_type2_10,c2t_distance_type2_11,c2t_distance_type2_12
0,"[71104.0, 70080.0, 67968.0, 67648.0, 65440.0, ...",71104.0,,,,,,,,,,
1,"[68800.0, 68352.0, 65856.0, 65600.0, 62720.0, ...",,,,,,,,,,,
2,"[68864.0, 68416.0, 65920.0, 65728.0, 62816.0, ...",,,,,,,,,,,
3,"[68800.0, 68352.0, 65856.0, 65600.0, 62720.0, ...",,,,,,,,,,,
4,"[68800.0, 68352.0, 65856.0, 65600.0, 62752.0, ...",,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
57351,"[23440.0, 23024.0, 22720.0, 22784.0, 23728.0, ...",,,,,,,,,,,
57352,"[23056.0, 22640.0, 22336.0, 22400.0, 23344.0, ...",,,,,,,,,,,
57353,"[23088.0, 22672.0, 22384.0, 22432.0, 23392.0, ...",,,,,,,,,,,
57354,"[23056.0, 22656.0, 22352.0, 22416.0, 23360.0, ...",,,,,,,,,,,


In [13]:
for id, row in tmp_df.iterrows(): # id: 0~57356
    for idx, value in enumerate(row): # idx: 0~10
        if isinstance(value,str):
            continue
        if np.any(pd.isnull(value)):
            tmp_df.iloc[id,(counter*(idx+1))] = np.nan
        else:
            tmp = value
            tmp_df.iloc[id,(counter*(idx+1))] = tmp[idx]

IndexError: invalid index to scalar variable.

In [33]:
all_df.iloc[0,2] = 5.0

In [34]:
all_df

,c2t_distance_type2,c2t_distance_type2_2,c2t_distance_type2_3,c2t_distance_type2_4,c2t_distance_type2_5,c2t_distance_type2_6,c2t_distance_type2_7,c2t_distance_type2_8,c2t_distance_type2_9,c2t_distance_type2_10,...,c2t_distance_type11_4,c2t_distance_type11_5,c2t_distance_type11_6,c2t_distance_type11_7,c2t_distance_type11_8,c2t_distance_type11_9,c2t_distance_type11_10,c2t_distance_type11_11,c2t_distance_type11_12,10
0,"[71104.0, 70080.0, 67968.0, 67648.0, 65440.0, ...",,5.0,,,,,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[68800.0, 68352.0, 65856.0, 65600.0, 62720.0, ...",,,,,,,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[68864.0, 68416.0, 65920.0, 65728.0, 62816.0, ...",,,,,,,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[68800.0, 68352.0, 65856.0, 65600.0, 62720.0, ...",,,,,,,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[68800.0, 68352.0, 65856.0, 65600.0, 62752.0, ...",,,,,,,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,,NaN
57352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,,NaN
57353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,,NaN
57354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,,NaN


In [24]:
all_df.to_excel(r"C:\Users\Kevin\Desktop\test.xlsx")

In [ ]:
pd.concat([expanded_df,all_df])

### Then calculate CV of all of the values to complete 1090 features: